## Generate Question-Answer Pairs

You will take the chunks of text from the previous step and generate question-answer pairs based on the content of each chunk.

Each chunk can be associated to one or more questions. The questions will be generated by an LLM, but you will need to make sure that the questions are relevant to the content of the chunk.

## Qwen3-30B-A3B-Instruct-2507

Download the model in `GGUF` format from [Hugging Face](https://huggingface.co/unsloth/Qwen3-30B-A3B-Instruct-2507-GGUF/tree/main). This requires more resources, so make sure you have enough RAM and VRAM.

Download one of the quantized models, I chose the following:

- `Qwen3-30B-A3B-Instruct-2507-Q4_K_M.gguf`

After that serve the model using [llama-server](https://github.com/ggml-org/llama.cpp/blob/master/tools/server/README.md):


```bash
llama-server \
  --model ~/.cache/llama.cpp/Qwen3-30B-A3B-Instruct-2507-Q4_K_M.gguf \
  --n-gpu-layers 18 \
  --ctx-size 4096 \
  --port 36912
```

### Create the system prompt

Start by creating the system prompt to teach the LLM how to generate questions given a chunk of text.

In [1]:
system_prompt = """
## Task description

You are an expert at generating relevant questions given some chunk of text. Your task is to read that text and create one or more questions that accurately reflect the key points, or information contained within that chunk.

This task is important for creating question-answer pairs that will be used to benchmark embedding models.

## Output format

You will output a JSON array of objects. Each object will have the following structure:

{
    "question": "A question generated from the chunk of text.",
}

## Examples

### Example 1

#### Text:

bitsandbytes enables accessible large language models via k-bit quantization for PyTorch. bitsandbytes provides three main features for dramatically reducing memory consumption for inference and training:

- 8-bit optimizers uses block-wise quantization to maintain 32-bit performance at a small fraction of the memory cost.
- LLM.int8() or 8-bit quantization enables large language model inference with only half the required memory and without any performance degradation. This method is based on vector-wise quantization to quantize most features to 8-bits and separately treating outliers with 16-bit matrix multiplication.
- QLoRA or 4-bit quantization enables large language model training with several memory-saving techniques that don’t compromise performance. This method quantizes a model to 4-bits and inserts a small set of trainable low-rank adaptation (LoRA) weights to allow training.

#### Output:

[
    {
        "question": "What is the primary purpose of the bitsandbytes library?"
    },
    {
        "question": "What are the three main features bitsandbytes provides for reducing memory consumption?"
    },
    {
        "question": "How does the LLM.int8() feature work to enable 8-bit inference without performance degradation?"
    },
    {
        "question": "What is QLoRA and how does it utilize 4-bit quantization for training?"
    },
    {
        "question": "What quantization technique do the 8-bit optimizers in bitsandbytes use?"
    },
    {
        "question": "Why is vector-wise quantization used in the LLM.int8() method, and how are outliers handled?"
    }
]

### Example 2

#### Text:

Creating your first star trail image
How it works

You’ve captured hundreds of photos, and now it’s time to blend them together to create your first star trail image. Each photo can be thought of as having two parts: the stars and the background.

The background remains still, while the stars appear to move from frame to frame due to Earth’s rotation. Our goal is to keep the background consistent while revealing the stars’ motion across the sky.

To do this, we gradually blend the images together to trace the path of each star. The pixels representing the background are mostly dark, while the ones representing the stars are bright.

We use the lighten blending mode for this process, it takes the brighter value between a pixel in one image and the corresponding pixel in the next. This simple rule creates the illusion of continuous trails.

If you can’t quite visualize it yet, don’t worry, I’ve included some illustrations to show exactly how this blending algorithm works.

To demonstrate how the lighten blending mode works, I created five small 8×8 images. Each cell is colored either black (background) or white (stars).

In the white cells, I’ve placed the number 1, which represents full brightness. In practice, each pixel in an 8-bit image stores a value between 0 and 255, where 255 corresponds to the maximum brightness. So, in this simplified example, 1 stands for 255.

The black cells correspond to a brightness value of 0. For clarity, they are left empty in the diagram because they are greater in number than the white cells.

In the first image, there are three stars. From one frame to the next, each star moves one pixel to the right and one pixel down. By the final frame, only one star remains visible, as the other two have moved outside the 8×8 grid.

Now, let’s apply the lighten blending mode to the first two images. In the illustration, you’ll see them represented as inputs to the max() function. This function compares the pixel values from both images and keeps the brighter one for each position. The resulting image is a blend of the two, showing all the stars that were visible in either frame.

The blended image becomes the new input for the next iteration of the max() function. The third image is then used as the second argument. Blend these two together, and repeat the process with the remaining images until you reach the fifth one.

The final blended result is your complete star trail image, showing the continuous paths traced by the stars over time.

#### Output:

[
    {
        "question": "What is the primary goal when blending photos to create a star trail image?"
    },
    {
        "question": "Why do the stars appear to move from one photo to the next?"
    },
        {
        "question": "What are the two main components of each image used to create a star trail photo?"
    },
    {
        "question": "What specific blending mode is used to create the star trail effect?"
    },
    {
        "question": "In the simplified 8×8 grid example, what do the white cells and black cells represent in terms of pixel brightness?"
    },
    {
        "question": "What mathematical function is used in the illustration to represent the lighten blending mode?"
    },
    {
        "question": "How does on a pixel level?"
    },
    {
        "question": "Describe the iterative process of blending the images to create the final star trail."
    },
]

## Notes

- Don't generate questions that are similar to each other.
- Questions should be relevant to the content of the chunk of text.
- Generate only a JSON array as specified in the output format.
"""

### Create a Pydantic object

We use a Pydantic object to define the structure of the output we want from the LLM. This will force the LLM to return the output in a structured format that we can easily parse.

In [3]:
from pydantic import BaseModel


class Question(BaseModel):
    question: str

### Use Qwen3

Create a few functions to generate questions using the Qwen3 model, create the user prompt, and parse the output. 

In [15]:
import json
import requests

# Don't forget to start the llama.cpp server!
LLAMA_SERVER_URL = "http://localhost:36912"


def generate_questions(
    system_prompt: str, user_prompt: str, max_tokens: int = 4096
) -> list[Question]:
    url = f"{LLAMA_SERVER_URL}/v1/chat/completions"
    headers = {"Content-Type": "application/json"}

    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {"role": "user", "content": user_prompt},
    ]

    data = {
        "messages": messages,
        "max_tokens": max_tokens,
    }

    try:
        response = requests.post(url=url, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        response_json = response.json()

        assistant_message = response_json["choices"][0]["message"]["content"].strip()
        parsed_questions = parse_questions(assistant_message)
        return parsed_questions

    except Exception as e:
        print(f"An error occurred: {e}")
        return []


def get_user_prompt(text_chunk: str) -> str:
    return f"""Here is a chunk of text from a document:

{text_chunk}

Please generate relevant questions based on the content of this text chunk.
"""


def parse_questions(raw_questions: str) -> list[Question]:
    data = json.loads(raw_questions)
    questions = [Question(**item) for item in data]
    return questions

Load the text chunks from the JSON file that you created in the previous step.

In [7]:
import json

file_path = "../data/chunks/rog_strix_gaming_notebook_pc_unscanned_file_chunks.json"
with open(file_path, "r") as f:
    text_chunks = json.load(f)

len(text_chunks)

43

Loop through each text chunk and generate questions using the function you created.

In [18]:
from tqdm import tqdm

question_answer_pairs = []
for text_chunk_object in tqdm(iterable=text_chunks, total=len(text_chunks)):
    text_chunk = text_chunk_object["text_chunk"]
    user_prompt = get_user_prompt(text_chunk)
    questions = generate_questions(system_prompt=system_prompt, user_prompt=user_prompt)
    if not questions:
        continue

    question_answer_pairs.extend(
        [
            {"chunk_id": text_chunk_object["id"], "question": question_object.question}
            for question_object in questions
        ]
    )

100%|██████████| 43/43 [10:21<00:00, 14.45s/it]

An error occurred: 400 Client Error: Bad Request for url: http://localhost:36912/v1/chat/completions


In [19]:
len(question_answer_pairs)

385

### Save the question-answer pairs

In [ ]:
import os
import json

output_directory = "../data/question_answer_pairs/"
os.makedirs(output_directory, exist_ok=True)

file_name = file_path.split("/")[-1].replace(".json", "_qa_pairs.json")
output_file_path = os.path.join(output_directory, file_name)

with open(output_file_path, "w") as f:
    json.dump(question_answer_pairs, f, indent=2)